## CrawlData

In [ ]:
!pip install selenium beautifulsoup4

In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument(f'user-agent=Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166')
options.add_argument("--disable-dev-shm-usage")


driver = webdriver.Chrome(options=options)

In [ ]:
data = []

for i in range(61, 100):
  url = f"https://www.newegg.com/tools/laptop-finder?page={i}"

  driver.get(url)
  # Scroll down to load all pages (dynamic loading)
  last_height = driver.execute_script("return document.body.scrollHeight")

  while True:
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
      new_height = driver.execute_script("return document.body.scrollHeight")
      if new_height == last_height:
          break
      last_height = new_height

  # Locate product containers
  soup = BeautifulSoup(driver.page_source, "lxml")

  product_containers = soup.find_all("tr")

  # Extract data for each product
  for container in product_containers:
      try:
          # Extract title and link from the anchor tag
          title_anchor = container.find("a", class_="goods-info")
          title = title_anchor.text.strip()
          link = title_anchor["href"]

          # Extract price from the price element
          price_element = container.find("div", class_="goods-price-current")
          price = price_element.find("span", class_="goods-price-value").text.strip()

          # Extract other data points from their respective td elements
          specs = {}
          for spec_element in container.find_all("td", class_="td-spec"):
              label = spec_element.find("div", class_="hid-text").text.strip()
              value = spec_element.find("span").text.strip()
              specs[label] = value

          # Create a dictionary for each product
          data.append({
              "title": title,
              "link": link,
              "price": price,
              "specs": specs,
          })
      except:
          continue

  # Print extracted data
  print(len(data))
  sleep(30)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780


In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df

,title,link,price,specs
0,"DELL E7470 14.0"" Grade B Laptop Intel Core i5 ...",https://www.newegg.com/dell-e7470/p/N82E168349...,485.99,"{'Screen Size': '14.0""', 'CPU type': 'Intel Co..."
1,"DELL 5480 14.0"" Grade A Laptop Intel Core i5 6...",https://www.newegg.com/dell-5480/p/N82E1683494...,309.99,"{'Screen Size': '14.0""', 'CPU type': 'Intel Co..."
2,"HP 15 G2 15.6"" Grade A Laptop Intel Core i7 4t...",https://www.newegg.com/hp-zbook-15-g2/p/N82E16...,619.99,"{'Screen Size': '15.6""', 'CPU type': 'Intel Co..."
3,"DELL E5570 15.6"" Grade A Laptop Intel Core i5 ...",https://www.newegg.com/dell-e5570/p/N82E168349...,418.99,"{'Screen Size': '15.6""', 'CPU type': 'Intel Co..."
4,Refurbished A GRADE Dell E7470 Core i5-6200U 2...,https://www.newegg.com/dell-e7470/p/N82E168348...,518.99,"{'Screen Size': '14.0""', 'CPU type': 'Intel Co..."
...,...,...,...,...
775,"Lenovo ThinkPad T14s Gen 3 21CQ004SUS 14"" Note...",https://www.newegg.com/p/1TS-000E-18KJ8?Item=1...,"1,909.99","{'Screen Size': '14.0""', 'Resolution': '1920 x..."
776,"HP EliteBook 630 G10 13.3"" Notebook - Full HD ...",https://www.newegg.com/p/1TS-000D-1C720?Item=1...,"1,205.99","{'Screen Size': '13.3""', 'Resolution': '1920 x..."
777,"Dell Precision 3000 3480 14"" Mobile Workstatio...",https://www.newegg.com/p/1TS-000A-0Y3K0?Item=1...,"1,993.99","{'Screen Size': '14.0""', 'Resolution': '1920 x..."
778,"Dell Precision 5000 5680 16"" Mobile Workstatio...",https://www.newegg.com/p/1TS-000A-0Z8J6?Item=1...,"3,499.99","{'Screen Size': '16.0""', 'Resolution': '1920 x..."


In [ ]:
features_d = dict(df['specs'])
new_fs = pd.DataFrame.from_dict(features_d).T
result = pd.concat([df, new_fs], axis=1)

In [ ]:
result = result.drop('specs', axis=1)

In [ ]:
result.isnull().sum()

title                 0
link                  0
price                 0
Screen Size          25
CPU type            298
Memory              265
Storage             272
Resolution           85
CPU Speed           364
Number of Cores     320
Operating System    294
Webcam              172
Touchscreen         169
Display Type        426
Graphic Type        124
GPU                 315
Color               505
Weight              566
Backlit Keyboard    675
Thunderbolt         704
Card Reader         741
dtype: int64

In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             780 non-null    object
 1   link              780 non-null    object
 2   price             780 non-null    object
 3   Screen Size       755 non-null    object
 4   CPU type          482 non-null    object
 5   Memory            515 non-null    object
 6   Storage           508 non-null    object
 7   Resolution        695 non-null    object
 8   CPU Speed         416 non-null    object
 9   Number of Cores   460 non-null    object
 10  Operating System  486 non-null    object
 11  Webcam            608 non-null    object
 12  Touchscreen       611 non-null    object
 13  Display Type      354 non-null    object
 14  Graphic Type      656 non-null    object
 15  GPU               465 non-null    object
 16  Color             275 non-null    object
 17  Weight          

In [ ]:
result.to_csv("dataa.csv")

# Xử lý cơ bản dữ liệu

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('newdataset 2.csv')

In [ ]:
df = df.drop('Unnamed: 0', axis=1)
df

,title,link,price,Screen Size,CPU type,Memory,Storage,GPU,Resolution,Weight,...,Touchscreen,CPU Speed,Number of Cores,Color,Display Type,Graphic Type,Operating System,Webcam,Card Reader,Thunderbolt
0,"MSI Modern 15H 15.6"" Ultra Thin and Light Prof...",https://www.newegg.com/classic-black-msi-moder...,699,"15.6""",Intel Core i7 13th Gen,32GB,1 TB PCIe,Intel Iris Xe Graphics,1920 x 1080,3 - 3.9 lbs.,...,No,13620H (2.40GHz),10-core (6P+4E) Processor,Black,Full HD,Integrated Card,Windows 11 Home,Yes,Micro SD,NaN
1,Dell Latitude 7400 Intel Core i7 8th Gen 8665U...,https://www.newegg.com/black-dell-latitude-740...,314.99,"14.0""",Intel Core i7 8th Gen,32GB,256 GB,Intel UHD Graphics 620,1920 x 1080,3 - 3.9 lbs.,...,Yes,8665U (1.90GHz),Quad-core Processor,Black,Full HD,Integrated Card,Windows 11 Pro,NaN,NaN,NaN
2,"Lenovo 83B40000US 15.6"" Laptop Intel Core i3 1...",https://www.newegg.com/abyss-blue-lenovo-ideap...,329.99,"15.6""",Intel Core i3 13th Gen,8GB,256 GB PCIe,Intel UHD Graphics,1920 x 1080,3 - 3.9 lbs.,...,No,1315U (1.20GHz),6-core (2P+4E) Processor,Blue,Full HD,Integrated Card,Windows 11 in S mode,Yes,SD Card Reader,NaN
3,"Lenovo 82XG0019US 16.0"" Laptop AMD Ryzen 7 700...",https://www.newegg.com/abyss-blue-lenovo-ideap...,649.99,"16.0""",AMD Ryzen 7 7000 Series,16GB,512 GB PCIe,AMD Radeon Graphics,1920 x 1200,4 - 4.9 lbs.,...,Yes,7730U (2.00GHz),8-core Processor,Blue,WUXGA,Integrated Card,Windows 11 Home,Yes,Micro SD,NaN
4,"GIGABYTE G6 KF - 16"" FHD 1920x1200 165Hz - NVI...",https://www.newegg.com/black-gigabyte-g6-kf-h3...,899.99,"16.0""",Intel Core i7 13th Gen,16GB,512 GB PCIe,NVIDIA GeForce RTX 4060 Laptop GPU,1920 x 1200,5 - 5.9 lbs.,...,No,13620H (2.40GHz),10-core (6P+4E) Processor,Black,WUXGA,Dedicated Card,Windows 11 Home,NaN,Micro SD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1923,"Lenovo ThinkPad T14s Gen 3 21CQ004SUS 14"" Note...",https://www.newegg.com/p/1TS-000E-18KJ8?Item=1...,"1,909.99","14.0""",NaN,NaN,NaN,NaN,1920 x 1200,NaN,...,NaN,NaN,NaN,Gray,NaN,Integrated Card,NaN,Yes,NaN,NaN
1924,"HP EliteBook 630 G10 13.3"" Notebook - Full HD ...",https://www.newegg.com/p/1TS-000D-1C720?Item=1...,"1,205.99","13.3""",NaN,NaN,NaN,NaN,1920 x 1080,NaN,...,No,NaN,NaN,NaN,NaN,Integrated Card,NaN,Yes,NaN,NaN
1925,"Dell Precision 3000 3480 14"" Mobile Workstatio...",https://www.newegg.com/p/1TS-000A-0Y3K0?Item=1...,"1,993.99","14.0""",NaN,NaN,NaN,NaN,1920 x 1080,NaN,...,No,NaN,NaN,Gray,NaN,Integrated Card,NaN,Yes,NaN,NaN
1926,"Dell Precision 5000 5680 16"" Mobile Workstatio...",https://www.newegg.com/p/1TS-000A-0Z8J6?Item=1...,"3,499.99","16.0""",NaN,NaN,NaN,NaN,1920 x 1200,NaN,...,No,NaN,NaN,Gray,NaN,Integrated Card,NaN,Yes,NaN,NaN


In [ ]:
df.isnull().sum()

title                  0
link                   0
price                  0
Screen Size           54
CPU type             475
Memory               390
Storage              422
GPU                  562
Resolution           183
Weight              1127
Backlit Keyboard    1415
Touchscreen          308
CPU Speed            602
Number of Cores      583
Color                949
Display Type         828
Graphic Type         279
Operating System     439
Webcam               431
Card Reader         1683
Thunderbolt         1618
dtype: int64

In [ ]:
df.fillna(np.nan, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             1928 non-null   object
 1   link              1928 non-null   object
 2   price             1928 non-null   object
 3   Screen Size       1874 non-null   object
 4   CPU type          1453 non-null   object
 5   Memory            1538 non-null   object
 6   Storage           1506 non-null   object
 7   GPU               1366 non-null   object
 8   Resolution        1745 non-null   object
 9   Weight            801 non-null    object
 10  Backlit Keyboard  513 non-null    object
 11  Touchscreen       1620 non-null   object
 12  CPU Speed         1326 non-null   object
 13  Number of Cores   1345 non-null   object
 14  Color             979 non-null    object
 15  Display Type      1100 non-null   object
 16  Graphic Type      1649 non-null   object
 17  Operating Syst

## Xử lý Resolution

In [ ]:
column = df['Resolution'].unique()
column

array(['1920 x 1080', '1920 x 1200', '2560 x 1600', '1366 x 768',
       '2560 x 1440', '2560 x 1600 (2.5K)', '2880 x 1620', '3200 x 2000',
       '2880 x 1800', nan, '3840 x 2400', '1600 x 900', '1440 x 900',
       '3840 x 2160', '3072 x 1920', '2496 x 1664', '1920 x 1280',
       '1920 x 1028', '2256 x 1504', '2240 x 1400', '2400 x 1600',
       '1360 x 768', '2560 x 1664', '1920*1080', '2736 x 1824',
       '3000 x 2000', '1800 x 1200', '3300 x 2200', '2880 x 1920',
       '3456 x 2160', '3240 x 2160', '2800 x 1752', '1280 x 720',
       '1536 x 1024', '2800x1620', '1366 x 912', '1920x1080', '1366x768'],
      dtype=object)

In [ ]:
def process_brand(brand):
  # Chuyển đổi thành chữ thường và loại bỏ khoảng trắng ở đầu và cuối chuỗi
  brand = str(brand).replace(" ","").replace("*", "x").replace("(2.5K)","")

  return brand

In [ ]:
df['Resolution'] = df['Resolution'].apply(process_brand)

In [ ]:
column = df['Resolution'].unique()
column

array(['1920x1080', '1920x1200', '2560x1600', '1366x768', '2560x1440',
       '2880x1620', '3200x2000', '2880x1800', 'nan', '3840x2400',
       '1600x900', '1440x900', '3840x2160', '3072x1920', '2496x1664',
       '1920x1280', '1920x1028', '2256x1504', '2240x1400', '2400x1600',
       '1360x768', '2560x1664', '2736x1824', '3000x2000', '1800x1200',
       '3300x2200', '2880x1920', '3456x2160', '3240x2160', '2800x1752',
       '1280x720', '1536x1024', '2800x1620', '1366x912'], dtype=object)

## Xử lý Screen Size

In [ ]:
column = df['Screen Size'].unique()
column

array(['15.6"', '14.0"', '16.0"', '13.4"', '17.3"', '17.0"', nan, '13.3"',
       '14.5"', '15.0"', '10.5"', '13.5"', '14.1"', 'No', '18.0"',
       '15.6" 4K / UHD', '14.4"', '11.6"', '17.3" 4K / UHD', '13.0"',
       '12.5"', '12.2"', '13.9"', '15.6', '16.1"', '13"', '12.3"',
       '12.4"', '10.1"', '14"', '17.3', '14', '12.0"', '11.0"', '12.5',
       '11.5"', '8.0"'], dtype=object)

In [ ]:
def process_screen(SS):
  if "No" in str(SS):
    return np.NaN
  elif "nan" in str(SS):
    return np.NaN
  else:
    SS = str(SS).split('"')
    SS =  SS[0]
    return SS

In [ ]:
df['Screen Size'] = df['Screen Size'].apply(process_screen)

In [ ]:
column = df['Screen Size'].unique()
column

array(['15.6', '14.0', '16.0', '13.4', '17.3', '17.0', nan, '13.3',
       '14.5', '15.0', '10.5', '13.5', '14.1', '18.0', '14.4', '11.6',
       '13.0', '12.5', '12.2', '13.9', '16.1', '13', '12.3', '12.4',
       '10.1', '14', '12.0', '11.0', '11.5', '8.0'], dtype=object)

## Xử lý Memory

In [ ]:
column = df['Memory'].unique()
column

array(['32GB', '8GB', '16GB', '4GB', '64GB', nan, '12GB',
       '16GB LPDDR5 RAM', '6GB', '160GB', '3GB'], dtype=object)

In [ ]:
def process_Memory(data):
  data = str(data)
  data = data.split("G")
  return data[0]

In [ ]:
df['Memory'] = df['Memory'].apply(process_Memory)

In [ ]:
df['Memory'] = df['Memory'].replace('nan', np.NaN)

In [ ]:
column = df['Memory'].unique()
column

array(['32', '8', '16', '4', '64', nan, '12', '6', '160', '3'],
      dtype=object)

In [ ]:
df['Memory'].fillna(0, inplace=True)
df['Memory'] = df['Memory'].astype(int)

## Xử lý Price

In [ ]:
column = df['price'].unique()
column

array(['699', '314.99', '329.99', '649.99', '899.99', '799', '1,699.99',
       '1,199.99', '379', '979.99', '209.99', '479', '949.99', '2,199.99',
       '379.99', '2,119.99', '2,799.99', '1,349.99', '1,289.99',
       '1,194.99', '719.99', '413.99', '259.99', '189.99', '629',
       '1,549.99', '269.99', '1,399.00', '529.99', '674.99', '679.99',
       '1,599.99', '1,299.00', '234.99', '1,649.99', '3,372.99', '729.99',
       '2,073.99', '2,458.99', '229.99', '2,756.99', '412.99', '2,654.99',
       '2,599.00', '794.99', '899', '799.99', '3,601.99', '399.99',
       '1,435.99', '1,299.99', '519.99', '1,399.99', '849.99', '933.99',
       '1,654.99', '929.99', '999.99', '1,440.00', '370.99', '3,699.99',
       '1,993.99', '3,799.99', '2,089.99', '950.99', '349.99', '749',
       '1,338.99', '1,579.99', '539.99', '1,144.99', '1,360.00',
       '1,170.00', '555.99', '2,609.99', '1,625.99', '2,799.00', '483.99',
       '2,199.00', '1,899.00', '559.99', '2,603.99', '1,099.99',
       '4,1

In [ ]:
def process_price(data):
  data = str(data).replace(",", "")
  return data

In [ ]:
df['price'] = df['price'].apply(process_price)

In [ ]:
df['price'] = df['price'].astype(float)

In [ ]:
df['price'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1928 entries, 0 to 1927
Series name: price
Non-Null Count  Dtype  
--------------  -----  
1928 non-null   float64
dtypes: float64(1)
memory usage: 15.2 KB


## Xử lý Storage

In [ ]:
column = df['Storage'].unique()
column

array(['1 TB PCIe', '256 GB', '256 GB PCIe', '512 GB PCIe', '512 GB',
       '64 GB', '2 TB PCIe', '128 GB', '1 TB', nan, '3 TB', '32 GB',
       '128 GB PCIe', '250 GB', '500 GB PCIe', '2 TB', '500 GB', '8 GB',
       '32 GB Flash Memory', '32 GB Flash', '180 GB',
       '128GB Flash Memory', '256GB PCIe 4.0 SSD', '16 GB', '1000', 'No',
       '512', '480', 'Flash Memory Capacity:   32 GB', '64 GB Flash',
       '512GB M.2 SSD', '120 GB', '256', '240 GB', '512GB PCIe SSD'],
      dtype=object)

In [ ]:
import re
def process_storage(data):
  data = str(data)
  if "TB" in data:
    data = data.split('TB')
    data = re.sub(r'\D', '', data[0])
    data = int(data) * 1024
    return data
  elif "GB" in data:
    data = data.split('GB')
    data = re.sub(r'\D', '', data[0])
    return data
  elif "No" in data:
    return 0
  else:
    return data

In [ ]:
df['Storage'].fillna(0, inplace=True)
df['Storage'].replace('No', 0, inplace=True)
df['Storage'] = df['Storage'].apply(process_storage)

In [ ]:
df['Storage'] = df['Storage'].astype(int)

In [ ]:
column = df['Storage'].unique()
column

array([1024,  256,  512,   64, 2048,  128,    0, 3072,   32,  250,  500,
          8,  180,   16, 1000,  480,  120,  240])

## Xử lý Backlit Keyboard

In [ ]:
column = df['Backlit Keyboard'].unique()
column

array(['Backlit', nan, 'RGB', 'Non-backlit', 'White Light', 'No',
       'Full Size'], dtype=object)

In [ ]:
df['Backlit Keyboard'].replace("No", "Non-backlit", inplace = True)
df['Backlit Keyboard'].fillna("Non-backlit", inplace = True)

In [ ]:
column = df['Backlit Keyboard'].unique()
column

array(['Backlit', 'Non-backlit', 'RGB', 'White Light', 'Full Size'],
      dtype=object)

## Xử lý Touchscreen

In [ ]:
column = df['Touchscreen'].unique()
column

array(['No', 'Yes', nan, 'NO_TOUCH', 'Yes (10-point multi-touch)'],
      dtype=object)

In [ ]:
df['Touchscreen'].replace("NO_TOUCH", "No", inplace = True)
df['Touchscreen'].replace("Yes (10-point multi-touch)", "Yes", inplace = True)
df['Touchscreen'].fillna('No', inplace = True)

## Xử lý Webcam

In [ ]:
column = df['Webcam'].unique()
column

array(['Yes', nan, '720P HD', 'HD camera with IR function',
       '1080p Full HD front-facing camera\r\n8 MP 1080p Full HD rear- facing camera',
       'HD type (30fps@720p)',
       'Full HD front-facing Surface Studio Camera\r\n1080p Full HD camera with wide field of view',
       '1080p FHD Webcam', 'Yes, User-Facing: 1080p',
       'HP True Vision 720p HD camera with temporal noise reduction and integrated dual array digital microphones',
       'HD type 720p', '720p HD camera with temporal noise reduction',
       '720p HD camera;8 MP camera (rear-facing)', '720p HD camera',
       '5MP IR camera', 'Yes - 4K (rear), 1080p (front)', 'No', '720p HD',
       '5MP', '5MP camera', '720p HD Camera', '720p HD Webcam',
       'HP Webcam', '720P HD camera', 'HD Camera',
       'HP TrueVision HD Camera', 'IR HD',
       'HP TrueVision HD Camera with integrated dual array digital microphone',
       'HD Camera with integrated dual array digital microphone',
       '1080p 30 fps FHD camera +

In [ ]:
import re
def process_webcam(data):
  data = str(data)
  if "No" in data:
    return "No"
  else:
    return "Yes"

In [ ]:
df['Webcam'] = df['Webcam'].fillna("No")
df['Webcam'] = df['Webcam'].apply(process_webcam)

## Xử lý CPU speed

In [ ]:
column = df['CPU Speed'].unique()
column

array(['13620H (2.40GHz)', '8665U (1.90GHz)', '1315U (1.20GHz)',
       '7730U (2.00GHz)', '1280P (1.80GHz)', '13900HX (2.20GHz)',
       '13700H (2.40GHz)', '1215U (1.20GHz)', '7735U (2.70GHz)',
       '1.60GHz', '1195G7 (2.90GHz)', '7945HX (2.50GHz)',
       '1335U (1.30GHz)', '13980HX (2.20GHz)', '13700HX (2.10GHz)',
       '7735HS (3.20GHz)', '13420H (2.10GHz)', '8350U (1.70GHz)',
       'N4020 (1.10GHz)', '5500U (2.10GHz)', '12800HX (2.00GHz)',
       '1235U (1.30GHz)', '7530U (2.00GHz)', '8650U (1.90GHz)',
       '8365U (1.60GHz)', '13900H (2.60GHz)', '11400H (2.70GHz)',
       '12700H (2.30GHz)', 'N4500 (1.10GHz)', '2.60GHz',
       '1255U (1.70GHz)', '11800H (2.30GHz)', '7945HX3D (2.30GHz)',
       '1340P', nan, '6800HS (3.20GHz)', '2.30GHz', '12450H (2.00GHz)',
       '1.90GHz', '7940HS (4.00GHz)', '2.5GHz', '5825U (2.00GHz)',
       '12500H (2.50GHz)', '8505 (1.20GHz)', '1360P (2.20GHz)',
       '5350U (1.80GHz)', '3.00GHz', '13500H (2.60GHz)', '1.70GHz',
       '12650H (2.30

In [ ]:
def process_webcam(data):
  data = str(data)
  if ("(" and ")") in data:
    data = data.split("(")
    data[1] = data[1].replace(")", "")
    data[0] = data[0].replace(" ", "")
    return data
  elif "GHz" in data or "Ghz" in data or "kHz" in data or "Mhz" in data:
    newdata = ["NotAvai", data]
    return newdata
  else:
    newdata = [data, "NotAvai"]
    return newdata

In [ ]:
newdata = df['CPU Speed'].apply(process_webcam)
newdata

0       [13620H, 2.40GHz]
1        [8665U, 1.90GHz]
2        [1315U, 1.20GHz]
3        [7730U, 2.00GHz]
4       [13620H, 2.40GHz]
              ...        
1923       [nan, NotAvai]
1924       [nan, NotAvai]
1925       [nan, NotAvai]
1926       [nan, NotAvai]
1927       [nan, NotAvai]
Name: CPU Speed, Length: 1928, dtype: object

In [ ]:
newdata.head(20)

0      [13620H, 2.40GHz]
1       [8665U, 1.90GHz]
2       [1315U, 1.20GHz]
3       [7730U, 2.00GHz]
4      [13620H, 2.40GHz]
5       [1280P, 1.80GHz]
6     [13900HX, 2.20GHz]
7      [13700H, 2.40GHz]
8       [1215U, 1.20GHz]
9       [7735U, 2.70GHz]
10    [NotAvai, 1.60GHz]
11     [1195G7, 2.90GHz]
12     [13620H, 2.40GHz]
13      [8665U, 1.90GHz]
14     [7945HX, 2.50GHz]
15      [1335U, 1.30GHz]
16    [13980HX, 2.20GHz]
17    [13900HX, 2.20GHz]
18    [13700HX, 2.10GHz]
19    [13900HX, 2.20GHz]
Name: CPU Speed, dtype: object

In [ ]:
# Tạo DataFrame từ danh sách b
df['CPU model'] = [item[0] for item in newdata]
df['CPU speed'] = [item[1] for item in newdata]

In [ ]:
column = df['CPU model'].unique()
column

array(['13620H', '8665U', '1315U', '7730U', '1280P', '13900HX', '13700H',
       '1215U', '7735U', 'NotAvai', '1195G7', '7945HX', '1335U',
       '13980HX', '13700HX', '7735HS', '13420H', '8350U', 'N4020',
       '5500U', '12800HX', '1235U', '7530U', '8650U', '8365U', '13900H',
       '11400H', '12700H', 'N4500', '1255U', '11800H', '7945HX3D',
       '1340P', 'nan', '6800HS', '12450H', '7940HS', '5825U', '12500H',
       '8505', '1360P', '5350U', '13500H', '12650H', '1355U', '7840HS',
       '1260P', '6800H', 'N5100', '9750H', '4980U', '8550U', '6900HS',
       '1185G7', '6300U', '1155G7', '12900H', '13950HX', '7300U', '1265U',
       '1240P', '5600H', '6900HX', '6600U', '1165G7', '4000+', '7600U',
       '1135G7', '4600M', '13800H', '1365U', '7820HQ', '7640HS', '5625U',
       '5900HX', '5250U', '1245U', '8265U', '7700HQ', '12900HK', '8250U',
       '10100Y', 'E3-1505Mv6', '1115G4', '1145G7', 'E-2276M', '9400H',
       '6200U', '8850H', '8565U', '9850H', 'N100', '6850U', '3250U',
    

In [ ]:
column = df['CPU speed'].unique()
column

array(['2.40GHz', '1.90GHz', '1.20GHz', '2.00GHz', '1.80GHz', '2.20GHz',
       '2.70GHz', '1.60GHz', '2.90GHz', '2.50GHz', '1.30GHz', '2.10GHz',
       '3.20GHz', '1.70GHz', '1.10GHz', '2.60GHz', '2.30GHz', 'NotAvai',
       '4.00GHz', '2.5GHz', '3.00GHz', '3.80GHz', '3.30GHz', '2.80GHz',
       '3.8GHz', '1.8 GHz', '1.00GHz', '4.30GHz', '3.49 GHz', '800 kHz',
       '3.4GHz', '1.30 GHz', '3.0GHz', '3.10GHz', '3.5GHz', '100 MHz',
       '2.0 GHz', '3.70GHz', 'Up to 4.4 GHz Performance Dual-Core',
       '1.6 GHz', '4.80 GHz', '3.10 GHz', '2.50 GHz', '3.60GHz',
       'Up to 3.6 GHz Efficiency 8-Core', '3.030GHz', '3.8 GHz', '1.7GHz',
       '4.3GHz', '3.40GHz', '2.8 GHz', '4.0GHz', '2.7GHz', '2.3GHz',
       '2.6GHz', '2.4GHz', '1.1GHz', '2.1GHz', '5.16GHz', '4 GHz'],
      dtype=object)

In [ ]:
import re
def process_cpuspeed(data):
  data = str(data).lower()
  if "ghz" in data:
    data = data.split("ghz")
    data[0] = re.sub(r'[^0-9.]', '', data[0])
    return data[0].replace(" ", "")
  elif "khz" in data:
    data = re.sub(r'[^0-9.]', '', data)
    return float(data)/10000000
  elif "mhz" in data:
    data = re.sub(r'[^0-9.]', '', data)
    return float(data)/1000

In [ ]:
df['CPU speed'] = df['CPU speed'].apply(process_cpuspeed)

In [ ]:
column = df['CPU speed'].unique()
column

array(['2.40', '1.90', '1.20', '2.00', '1.80', '2.20', '2.70', '1.60',
       '2.90', '2.50', '1.30', '2.10', '3.20', '1.70', '1.10', '2.60',
       '2.30', None, '4.00', '2.5', '3.00', '3.80', '3.30', '2.80', '3.8',
       '1.8', '1.00', '4.30', '3.49', 8e-05, '3.4', '3.0', '3.10', '3.5',
       0.1, '2.0', '3.70', '4.4', '1.6', '4.80', '3.60', '3.6', '3.030',
       '1.7', '4.3', '3.40', '2.8', '4.0', '2.7', '2.3', '2.6', '2.4',
       '1.1', '2.1', '5.16', '4'], dtype=object)

In [ ]:
column = df['CPU model'].unique()
column

array(['13620H', '8665U', '1315U', '7730U', '1280P', '13900HX', '13700H',
       '1215U', '7735U', 'NotAvai', '1195G7', '7945HX', '1335U',
       '13980HX', '13700HX', '7735HS', '13420H', '8350U', 'N4020',
       '5500U', '12800HX', '1235U', '7530U', '8650U', '8365U', '13900H',
       '11400H', '12700H', 'N4500', '1255U', '11800H', '7945HX3D',
       '1340P', 'nan', '6800HS', '12450H', '7940HS', '5825U', '12500H',
       '8505', '1360P', '5350U', '13500H', '12650H', '1355U', '7840HS',
       '1260P', '6800H', 'N5100', '9750H', '4980U', '8550U', '6900HS',
       '1185G7', '6300U', '1155G7', '12900H', '13950HX', '7300U', '1265U',
       '1240P', '5600H', '6900HX', '6600U', '1165G7', '4000+', '7600U',
       '1135G7', '4600M', '13800H', '1365U', '7820HQ', '7640HS', '5625U',
       '5900HX', '5250U', '1245U', '8265U', '7700HQ', '12900HK', '8250U',
       '10100Y', 'E3-1505Mv6', '1115G4', '1145G7', 'E-2276M', '9400H',
       '6200U', '8850H', '8565U', '9850H', 'N100', '6850U', '3250U',
    

## Xử lý Display Type

In [ ]:
column = df['Display Type'].unique()
column

array(['Full HD', 'WUXGA', nan, 'WQXGA', 'WQHD', 'QHD+ IPS (16:10)',
       'QHD', 'QHD+', 'HD', 'WQXGA+', 'Retina Display', 'UHD+', 'HD+',
       'TFT', 'WQUXGA', 'PixelSense Display', 'FHD+', 'UHD',
       '1920 x 1200', 'IPS', 'DUAL-MODE: UHD + 120Hz / FHD + 240Hz',
       'QHD+ (16 : 10)', 'FHD', 'Full HD Plus', 'LCD', 'LED', 'OLED',
       'PixelSense Flow Display', 'WVA Display', 'ComfyView', 'Wide UXGA',
       'Twisted nematic (TN)', 'CineCrystal', 'Wide XGA', 'OLED Display',
       'Vivid Color, Corning Gorilla glass 3, Color Calibrated, sRGB Color Space, PixelSense, Dolby Vision IQ',
       'LCD CineCrystal', 'IPS Display'], dtype=object)

## Xử lý Graphic Type

In [ ]:
column = df['Graphic Type'].unique()
column

array(['Integrated Card', 'Dedicated Card', nan, 'Dedicated',
       'Dedicated Graphics', 'Integrated Graphics', 'Integrated',
       'UHD Graphics', 'Shared', 'Intel HD Graphics 500'], dtype=object)

In [ ]:
def process_webcam(data):
  data = str(data)
  if "Integrated" in data or "Intel HD Graphics 500" in data or "UHD Graphics" in data:
    return "Integrated"
  elif "Dedicated" in data or "Shared" in data:
    return "Dedicated"
  else:
    return np.NaN

In [ ]:
df['Graphic Type'] = df['Graphic Type'].apply(process_webcam)

In [ ]:
df['Graphic Type'].isna().sum()

279

In [ ]:
for index, row in df.iterrows():
    if str(row['Graphic Type']) == "nan":
      if 'NVIDIA' in str(row['GPU']) or 'AMD Radeon' in str(row['GPU']) :
          df.at[index, 'Graphic Type'] = 'Dedicated'
      elif row['GPU'] != '':
          df.at[index, 'Graphic Type'] = 'Integrated'
      else:
          df.at[index, 'Graphic Type'] = np.NaN

## Xử lý Operating System

In [ ]:
column = df['Operating System'].unique()
column

array(['Windows 11 Home', 'Windows 11 Pro', 'Windows 11 in S mode',
       'Windows 10 Pro', 'Windows 10 Home', 'Chrome OS', nan, 'macOS',
       'macOS 10.12 Sierra', 'Windows 10', 'Windows 10 S',
       'Windows 10 in S mode', 'OS X 10.10 Yosemite', 'macOS 13 Ventura',
       'Windows 11', 'macOS 10.13 High Sierra',
       'Windows 11 Pro Education', 'Android 9.0 (Pie)', 'No',
       'macOS 10.15 Catalina'], dtype=object)

In [ ]:
import re
def process_OS(data):
    data = str(data)
    if "macOS" in data:
        return "macOS"
    elif "Windows" in data and "S" in data and "10" in data:
        return "Windows 10 in S mode"
    elif "Windows" in data and "S" in data and "11" in data:
        return "Windows 11 in S mode"
    elif data == "nan":
        return "No"
    else:
        return data

In [ ]:
df['Operating System'] = df['Operating System'].apply(process_OS)

## Xử lý Card Reader

In [ ]:
column = df['Card Reader'].unique()
column

array(['Micro SD', nan, 'SD Card Reader', 'SD 4.0 card reader', 'SD',
       '4-in-1', '1 x card reader (microSD) (UHS-II)', 'SD7.0',
       'Full Size SD Express', 'Multi-format SD media card reader',
       'SD\xa0Express\xa07.0\xa0card\xa0reader',
       'SDXC 7.0, PCIE interface, Full SD size connector',
       '1 x card reader (microSD) (UHS-II, 312MB/s)', 'UFS/Micro SD',
       'Full Size SD, Smart card Reader (CAC)', '2-in-1',
       'SD 7.0 Card Reader', 'SD7.0 (XC/HC)', 'MicroSDXC',
       'Hidden Micro SD', 'MicroSDXC Card Reader', 'SmartCard Reader',
       'SD UHS-III', '(SD Card, SDHC Card, SDXC Card)',
       'SD Card, SDHC Card, SDXC Card, SDUC Card',
       'UHS-II SD Card Reader',
       'Single Slot: microSD/microSDHC/microSDXC',
       '1 x microSD Card Reader with UHS-II support',
       '1 multi-format SD media card reader', '4-in-1 Card Reader',
       'SD Express 7.0 card reader', 'MicroSDXC card reader',
       'microSD Card Reader', 'MicroSD Multi-media Card Re

In [ ]:
import re
def process_cardreader(data):
    data = str(data)
    if ("No" in data) or data == "nan":
        return "No"
    else:
        return "Yes"

In [ ]:
df['Card Reader'] = df['Card Reader'].apply(process_cardreader)

## Xử lý Thunderbolt

In [ ]:
column = df['Thunderbolt'].unique()
column

array([nan, '1 x Thunderbolt 4', '1 x Thunderbolt 4 / USB4',
       '2 x Thunderbolt 4', '2 x Thunderbolt 4 / USB4',
       '2 x Thunderbolt 3',
       'Four Thunderbolt 3 (USB-C) ports with support for:\r\n\r\nCharging\r\nDisplayPort\r\nThunderbolt (up to 40Gb/s)\r\nUSB 3.1 Gen 2 (up to 10Gb/s)',
       '2 x USB-C® with USB4®/ Thunderbolt™ 4 (with DisplayPort and Power Delivery)',
       '1 x Thunderbolt', '1 x Thunderbolt (USB Type-C)',
       '2 x Thunderbolt 4 (Supports DisplayPort and 100 W Power Delivery)',
       '1 Thunderbolt™ 4 with USB Type-C® 40Gbps signaling rate (USB Power Delivery, DisplayPort™ 1.4)',
       '1 x Thunderbolt 3',
       '2 Thunderbolt™ 4 with USB4™ Type-C® 40Gbps signaling rate (USB Power Delivery, DisplayPort™ 1.4, HP Sleep and Charge)',
       '2 x Thunderbolt 4 supports display / power delivery',
       '2 Thunderbolt 4 (USB Type-C™) ports with DisplayPort and Power Delivery',
       '2 x Thunderbolt 4 (Supports DisplayPort and Power Delivery)'],
     

In [ ]:
import re
def process_Thunderbolt(data):
    data = str(data)
    if ("No" in data) or data == "nan":
        return "No"
    else:
        return "Yes"

In [ ]:
df['Thunderbolt'] = df['Thunderbolt'].apply(process_cardreader)

## Xử lý Weight

In [ ]:
column = df['Weight'].unique()
column

array([3.  , 4.  , 5.  ,  nan, 6.  , 5.4 , 2.  , 5.16, 4.63, 5.07, 5.7 ,
       2.2 , 5.51, 5.29, 1.15, 5.5 , 3.75, 3.44, 4.59, 6.39, 4.37, 5.24,
       3.99, 3.9 , 2.78, 4.43, 3.56, 3.42, 4.58, 3.53, 3.2 , 7.  , 6.06,
       2.86, 4.56, 5.1 , 4.8 , 2.8 , 3.3 , 3.92, 3.09, 2.94, 5.73, 2.75,
       4.7 , 4.6 , 2.59, 3.04, 3.7 , 3.03, 1.79, 3.76, 5.26, 5.25, 3.86,
       2.73, 3.24, 3.33, 3.85, 3.73, 3.05, 1.  , 3.97, 1.9 , 2.4 , 6.73,
       3.02, 4.5 , 4.4 , 3.21, 4.19, 4.84, 5.14, 2.93, 1.73, 3.12, 3.06,
       3.1 , 4.17, 2.97, 2.6 , 2.87, 3.37, 2.37, 3.84, 2.33, 3.81, 3.35,
       3.64, 3.39, 4.81, 5.84, 3.31, 3.41, 3.4 , 3.28, 3.22, 1.22, 5.18,
       6.28, 5.27, 5.09, 2.91, 4.05, 2.71, 3.88, 2.95, 2.62])

In [ ]:
def weight_avg(x):
    if str(x) == 'nan':
        return np.NaN
    x = str(x).split(' ')
    try:
        return float(x[0])
    except:
        try:
            return float(x[1])
        except:
            return np.NaN

In [ ]:
df['Weight'] = df['Weight'].apply(weight_avg)

In [ ]:
df.isnull().sum()

title                  0
link                   0
price                  0
Screen Size           88
CPU type             475
Memory                 0
Storage                0
GPU                  562
Resolution             0
Weight              1166
Backlit Keyboard       0
Touchscreen            0
CPU Speed            602
Number of Cores      583
Color                949
Display Type         828
Graphic Type           0
Operating System       0
Webcam                 0
Card Reader            0
Thunderbolt            0
CPU model              0
CPU speed            605
dtype: int64

In [ ]:
df.to_csv("newdataset 3.csv")

In [ ]:
df = pd.read_csv('newdataset 3.csv')

In [ ]:
df = df.drop(['CPU Speed', 'Number of Cores', 'Color', 'Display Type', 'CPU speed'], axis=1)

In [ ]:
df = df.dropna(subset=['CPU type'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1453 entries, 0 to 1920
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1453 non-null   int64  
 1   title             1453 non-null   object 
 2   link              1453 non-null   object 
 3   price             1453 non-null   float64
 4   Screen Size       1423 non-null   float64
 5   CPU type          1453 non-null   object 
 6   Memory            1453 non-null   int64  
 7   Storage           1453 non-null   int64  
 8   GPU               1291 non-null   object 
 9   Resolution        1400 non-null   object 
 10  Weight            721 non-null    float64
 11  Backlit Keyboard  1453 non-null   object 
 12  Touchscreen       1453 non-null   object 
 13  Graphic Type      1453 non-null   object 
 14  Operating System  1453 non-null   object 
 15  Webcam            1453 non-null   object 
 16  Card Reader       1453 non-null   object 
 17  

In [ ]:
df.to_csv("newdataset 4.csv")